In [ ]:
import pandas as pd
import numpy as np

In [ ]:
#read the file
horse_data = pd.read_csv('horse_data.csv', header = None)
horse_data.head()

,0,1,2,3,4,5,6,7,8,9,...,18,19,20,21,22,23,24,25,26,27
0,2,1,530101,38.50,66,28,3,3,?,2,...,45.00,8.40,?,?,2,2,11300,0,0,2
1,1,1,534817,39.2,88,20,?,?,4,1,...,50,85,2,2,3,2,2208,0,0,2
2,2,1,530334,38.30,40,24,1,1,3,1,...,33.00,6.70,?,?,1,2,0,0,0,1
3,1,9,5290409,39.10,164,84,4,1,6,2,...,48.00,7.20,3,5.30,2,1,2208,0,0,1
4,2,1,530255,37.30,104,35,?,?,6,2,...,74.00,7.40,?,?,2,2,4300,0,0,2


In [ ]:
#throw out unuseful columns
horse_data.drop(horse_data.columns[[2,7,8,9,11,12,13,14,15,16,17,18,19,20,21,23,24,25,26,27]],axis = 1, inplace = True)
horse_data.head()

,0,1,3,4,5,6,10,22
0,2,1,38.50,66,28,3,5,2
1,1,1,39.2,88,20,?,3,3
2,2,1,38.30,40,24,1,3,1
3,1,9,39.10,164,84,4,2,2
4,2,1,37.30,104,35,?,?,2


In [ ]:
#assign correct names
horse_data.columns = ['surgery?', 'Age', 'rectal temperature', 'pulse', 'respiratory rate', 'temperature of extremities', 'pain', 'outcome']
horse_data.head(15)

,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
0,2,1,38.50,66,28,3,5,2
1,1,1,39.2,88,20,?,3,3
2,2,1,38.30,40,24,1,3,1
3,1,9,39.10,164,84,4,2,2
4,2,1,37.30,104,35,?,?,2
5,2,1,?,?,?,2,2,1
6,1,1,37.90,48,16,1,3,1
7,1,1,?,60,?,3,?,2
8,2,1,?,80,36,3,4,3
9,2,9,38.30,90,?,1,5,1


Задание 2. Первичное изучение данных
Проанализировать значения по столбцам, рассчитать базовые статистики, найти выбросы.

In [ ]:
#Surgery metrics
horse_data['surgery?'].value_counts()
mask1 = horse_data['surgery?'] == '1'
who_had_surgery = horse_data['surgery?'][mask1].count()
percent_surgery = round(who_had_surgery*100/horse_data['surgery?'].count())
print(percent_surgery, '%', 'of all the horses had the surgery. It\'s more than half')

60 % of all the horses had the surgery. It's more than half


In [ ]:
#Age. As the info about dataset says, there is 2 possible outcomes -  1 or 2. Outcome 2 is missing so
#most probably 9 is a mistaken value. Let's replace it to 2.
horse_data['Age'].value_counts()
horse_data['Age'] = horse_data['Age'].replace(9,2)
horse_data.head()
mask2 = horse_data['Age'] == 2
young_horses = horse_data['Age'][mask2].count()
percent_young = round(young_horses*100/horse_data['Age'].count())
print('Only', percent_young, '%', 'of all the horses are young. So most of the sick horses are adults.')

Only 8 % of all the horses are young. So most of the sick horses are adults.


In [ ]:
#rectal temperature
horse_copy = horse_data.copy()
#there are many gaps, so we will try to see statistics without them
mask3 = horse_copy['rectal temperature'] != "?"
horse_copy = horse_copy[mask3]
#into float
horse_copy['rectal temperature'] = horse_copy['rectal temperature'].astype(float)
#average temperature is higher than the normal one
print('The mean is', horse_copy['rectal temperature'].mean())
#median is also higher than average and is close to mean
print('The median is', horse_copy['rectal temperature'].median())
#The most frequent value is also higher than normal
print('The mode is', horse_copy['rectal temperature'].mode()[0])
#standard deviation
print('The standard deviation is', horse_copy['rectal temperature'].std())
#dispertion
print('The variance is', horse_copy['rectal temperature'].var())
#quantiles
print(horse_copy['rectal temperature'].quantile()) #equal to median
q1 = horse_copy['rectal temperature'].quantile(0.25)
print('The first quartile is', q1) #low
q3 = horse_copy['rectal temperature'].quantile(0.75)
print('The third quartile is', q3)
#interquartile range
iqr = q3 - q1
print('The interquartile range is', iqr)
#searching outliers
#I don't think that we need to remove these outliers, because these are possible rectal temperatures.
#High temperatures more than 40 mean that the horse has a strong infection
#Temperatures around 36 and lower mean that these horses are in a late shock.
lower_bound = q1 - (2*iqr)
upper_bound = q3 + (2*iqr)
remove_outliers = horse_copy[~horse_copy['rectal temperature'].between (lower_bound,upper_bound, inclusive = True)]
display(remove_outliers)

#Analyzing high and low temperatures
#Number of horses with temperature higher than normal is 3 times higher than lower than normal
mask_low = horse_copy['rectal temperature']< 37.8
print(len(horse_copy[mask_low]))
mask_high = horse_copy['rectal temperature']> 37.8
print(len(horse_copy[mask_high]))
mask_normal = horse_copy['rectal temperature']== 37.8
print(len(horse_copy[mask_normal]))
#around 7% of horses have normal temperature
print('Percentage of horses who have normal temperature:', round(len(horse_copy[mask_normal])/len(horse_copy)*100), '%')

The mean is 38.16791666666667
The median is 38.2
The mode is 38.0
The standard deviation is 0.7322886641121578
The variance is 0.5362466875871685
38.2
The first quartile is 37.8
The third quartile is 38.5
The interquartile range is 0.7000000000000028


<ipython-input-10-7390a7a4f6a7>:33: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  remove_outliers = horse_copy[~horse_copy['rectal temperature'].between (lower_bound,upper_bound, inclusive = True)]


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
44,1,1,35.4,140,24,3,4,3
54,2,1,40.3,114,36,3,2,3
91,2,1,40.3,114,36,3,2,2
141,2,1,36.0,42,30,?,?,2
238,2,1,36.1,88,?,3,3,3
259,1,1,40.8,72,42,3,2,2
281,2,1,40.0,78,?,3,2,2


56
167
17
Percentage of horses who have normal temperature: 7 %


In [ ]:
#pulse
horse_copy2 = horse_data.copy()
#there are rather many gaps, so we will try to see statistics without them
mask3 = horse_copy2['pulse'] != "?"
horse_copy2 = horse_copy2[mask3]
#into float
horse_copy2['pulse'] = horse_copy2['pulse'].astype(float)
#average pulse is more than 2 times higher than the normal one
print('The mean is', horse_copy2['pulse'].mean())
#median is also higher2 times higher
print('The medianan is', horse_copy2['pulse'].median())
#The most frequent value is also higher than normal
print('The mode is', horse_copy2['pulse'].mode()[0])
#standard deviation
print('The standard deviation is', horse_copy2['pulse'].std())
#dispertion
print('The variance is', horse_copy2['pulse'].var())
#quantiles
print(horse_copy2['pulse'].quantile()) #equal to median
q1 = horse_copy2['pulse'].quantile(0.25)
print('The first quartile is', q1) #low
q3 = horse_copy2['pulse'].quantile(0.75)
print('The third quartile is', q3)
#interquartile range
iqr = q3 - q1
print('The interquartile range is', iqr)
#I don't think that we need to remove these outliers, because these is a possible pulse.
#Most probably the horse is close to death with such pulse
lower_bound = q1 - (1.5*iqr)
upper_bound = q3 + (1.5*iqr)
remove_outliers = horse_copy2[~horse_copy2['pulse'].between (lower_bound,upper_bound, inclusive = True)]
display(remove_outliers)

#Analyzing high and low pulses
mask_low2 = horse_copy2['pulse']< 30.0
print(len(horse_copy2[mask_low2]))
mask_high2 = horse_copy2['pulse']> 40.0
print(len(horse_copy2[mask_high2]))
mask_normal2 = horse_copy2['pulse']>=30 
mask_normal3 = horse_copy2['pulse']<=40
print(len(horse_copy2[mask_normal2][mask_normal3]))
#around 9% of horses have normal pulse
print('Percentage of horses who have normal pulse:', round(len(horse_copy2[mask_normal2&mask_normal3])/len(horse_copy2)*100), '%')
#There are no horses with low tempearture, that means that ore than 90% have a high temperature

The mean is 71.91304347826087
The medianan is 64.0
The mode is 48.0
The standard deviation is 28.630556660735
The variance is 819.7087747035573
64.0
The first quartile is 48.0
The third quartile is 88.0
The interquartile range is 40.0


<ipython-input-41-1b5261569608>:31: FutureWarning: Boolean inputs to the `inclusive` argument are deprecated infavour of `both` or `neither`.
  remove_outliers = horse_copy2[~horse_copy2['pulse'].between (lower_bound,upper_bound, inclusive = True)]


,surgery?,Age,rectal temperature,pulse,respiratory rate,temperature of extremities,pain,outcome
3,1,2,39.10,164.0,84,4,2,2
41,2,2,39.00,150.0,72,?,?,1
55,1,2,38.60,160.0,20,3,3,2
255,1,2,38.80,184.0,84,1,4,2
275,1,2,38.80,150.0,50,1,5,2


0
252
24
Percentage of horses who have normal temperature: 9 %


In [ ]:
#respiratory rate
horse_data['respiratory rate'].value_counts().head()
#There is a great percentage of the gaps, but info says that the usefulness is
#doubtful due to the great fluctuations. So we can skip this data and remove it during analyzing the gaps

?     58
20    28
24    27
16    22
30    19
Name: respiratory rate, dtype: int64

In [ ]:
#temperature of extrimities
horse_data['temperature of extremities'].value_counts()
horse_copy3 = horse_data.copy()
#there are many gaps, so we will try to see statistics without them
mask4 = horse_copy3['temperature of extremities'] != "?"
horse_copy3 = horse_copy3[mask4]
horse_copy3.head()
#The most frequent value is 'cool', meaning that most of the animals probably had possible shock
print('The mode is', horse_copy3['temperature of extremities'].mode()[0], 'which means "cool temperature"')
#We can work with the gaps by correlating them with the rectal temperature values

The mode is 3 which means "cool temperature"


In [ ]:
#pain
horse_data['pain'].value_counts()
horse_copy4 = horse_data.copy()
#there are many gaps, so we will try to see statistics without them
mask5 = horse_copy4['pain'] != "?"
horse_copy4 = horse_copy4[mask4]
horse_copy4.head()
#The most frequent value is 3, meaning that most of the animals probably have intermittent mild pain
print('The mode is', horse_copy4['pain'].mode()[0], 'which means "mild pain"')

#Analyzing strong pain and pain abcense
mask_alert = horse_copy4['pain'] == '1'
print(len(horse_copy4[mask_alert]))
mask_severe = horse_copy4['pain'] == '5'
print(len(horse_copy4[mask_severe]))
print('Percentage of horses who don\'t have pain:', round(len(horse_copy4[mask_alert])/len(horse_copy4)*100), '%')
print('Percentage of horses who have strongest pain:', round(len(horse_copy4[mask_severe])/len(horse_copy4)*100), '%')
#There are no horses with low tempearture, that means that ore than 90% have a high temperature

The mode is 3 which means "mild pain"
36
39
Percentage of horses who don't have pain: 15 %
Percentage of horses who have strongest pain: 16 %


In [ ]:
#outcome
horse_data['outcome'].value_counts()
#we see only one gap
#Most of the horses survived, 
#Analyzing strong pain and pain abcense
mask_lived = horse_data['outcome'] == '1'
print(len(horse_data[mask_lived]))
mask_died1 = horse_data['outcome'] == '2'
print(len(horse_data[mask_died1]))
mask_died2 = horse_data['outcome'] == '3'
print(len(horse_data[mask_died2]))
#Biggest part of horses remained alive
print('Percentage of horses who survived:', round(len(horse_data[mask_lived])/len(horse_data)*100), '%')
print('Percentage of horses who died or were euthonized:', round(len(horse_data[mask_died1|mask_died2])/len(horse_data)*100), '%')

178
77
44
Percentage of horses who survived: 59 %
Percentage of horses who died or were euthonized: 40 %


Задание 3. Работа с пропусками

Рассчитать количество пропусков для всех выбранных столбцов. Принять и обосновать решение о методе заполнения пропусков по каждому столбцу на основе рассчитанных статистик и возможной взаимосвязи значений в них. Сформировать датафрейм, в котором пропуски будут отсутствовать. **Текст, выделенный полужирным шрифтом**

In [ ]:
#respiratory rate
#First of all, we can delete this column, because it has many gaps and as the data says most probably is useless.
horse_data = horse_data.drop(['respiratory rate'], axis=1)

In [ ]:
#Now let's change all the question marks with NaN
horse_data = horse_data.replace('?',np.nan)
horse_data.head(10)

,surgery?,Age,rectal temperature,pulse,temperature of extremities,pain,outcome
0,2,1,38.50,66,3,5,2
1,1,1,39.2,88,NaN,3,3
2,2,1,38.30,40,1,3,1
3,1,2,39.10,164,4,2,2
4,2,1,37.30,104,NaN,NaN,2
5,2,1,NaN,NaN,2,2,1
6,1,1,37.90,48,1,3,1
7,1,1,NaN,60,3,NaN,2
8,2,1,NaN,80,3,4,3
9,2,2,38.30,90,1,5,1


In [ ]:
#If there are more than 3 gaps, let's delete these rows from analysis
horse_data = horse_data.dropna(thresh = 4)

In [ ]:
#surgery
#only one gap
mask = horse_data['surgery?'].isna()
h2 = horse_data[mask]
display(h2)
#We don't know if this horse had the surgery and whether it survived. So better to delete this row from our
#database, it won't have a critical influence
mask1 = horse_data['outcome'].isna()
h3 = horse_data[mask1]
display(h3)
#By the way, is it an only horse with the uncertain outcome, so we can delete this ror for sure
horse_data = horse_data[~horse_data['surgery?'].isna()]

,surgery?,Age,rectal temperature,pulse,temperature of extremities,pain,outcome
132,NaN,1,38.0,48,3,4,NaN


,surgery?,Age,rectal temperature,pulse,temperature of extremities,pain,outcome
132,NaN,1,38.0,48,3,4,NaN


In [ ]:
#Age. No gaps
horse_data['Age'].value_counts()

1    269
2     22
Name: Age, dtype: int64

In [ ]:
#rectal temperature
horse_data['rectal temperature'].value_counts()
#as data says, there is direct correlation between rectal temperature and temperature of extremities. So we can
#exchage the gaps of horse whose extremeties are warm with the temperature from 3rd quartile (extended)
horse_data['rectal temperature'] = horse_data['rectal temperature'].astype(float)
horse_data.loc[(horse_data['temperature of extremities']=='2')&(horse_data['rectal temperature'].isna()), 'rectal temperature'] = horse_data['rectal temperature'].quantile(0.75)
rect_gaps = horse_data[horse_data['rectal temperature'].isna()]
rect_gaps
#as soon as the temperature always changes, we can put the average temperature to all other horses without temperature
horse_data['rectal temperature']=horse_data['rectal temperature'].fillna(horse_data['rectal temperature'].mean())
horse_data.head()


,surgery?,Age,rectal temperature,pulse,temperature of extremities,pain,outcome
0,2,1,38.5,66,3,5,2
1,1,1,39.2,88,NaN,3,3
2,2,1,38.3,40,1,3,1
3,1,2,39.1,164,4,2,2
4,2,1,37.3,104,NaN,NaN,2


In [ ]:
#temperature of extremities
horse_data['temperature of extremities'].value_counts()
ext_gaps = horse_data[horse_data['temperature of extremities'].isna()]
#the high temperature of extremities correlates with the rectal temperature. So we can change gaps to categories
#depending on how high is the rectal temperature:
horse_data.loc[(horse_data['rectal temperature']>(horse_data['rectal temperature'].quantile(0.75)))&(horse_data['temperature of extremities'].isna()), 'temperature of extremities'] = '4'
horse_data.loc[(horse_data['rectal temperature']>(horse_data['rectal temperature'].mean()))&(horse_data['temperature of extremities'].isna()), 'temperature of extremities'] = '3'
horse_data.loc[horse_data['temperature of extremities'].isna(), 'temperature of extremities'] = '1'
horse_data.head()

,surgery?,Age,rectal temperature,pulse,temperature of extremities,pain,outcome
0,2,1,38.5,66,3,5,2
1,1,1,39.2,88,4,3,3
2,2,1,38.3,40,1,3,1
3,1,2,39.1,164,4,2,2
4,2,1,37.3,104,1,NaN,2


In [ ]:
#pulse
#We can find the medians according to the categories of 'pain' and put the medians accordingly instead of gaps:
pulse_gaps = horse_data[horse_data['pulse'].isna()]
horse_data['pulse'] = horse_data['pulse'].astype(float)
horse_data['pulse']=horse_data['pulse'].fillna(horse_data.groupby('pain')['pulse'].transform('median'))
horse_data.head()

,surgery?,Age,rectal temperature,pulse,temperature of extremities,pain,outcome
0,2,1,38.5,66.0,3,5,2
1,1,1,39.2,88.0,4,3,3
2,2,1,38.3,40.0,1,3,1
3,1,2,39.1,164.0,4,2,2
4,2,1,37.3,104.0,1,NaN,2


In [ ]:
#pain
pain_gaps = horse_data[horse_data['pain'].isna()]
#If there was no surgery and pulse is more than 3rd quantile we consider that the pain level is the highest
#If there was no surgery and pulse is more than median we consider that the pain level is high
#In all other cases we consider that the pain is mild
horse_data.loc[(horse_data['surgery?']=='2')&(horse_data['pulse']>(horse_data['pulse'].quantile(0.75)))&(horse_data['pain'].isna()), 'pain'] = '5'
horse_data.loc[(horse_data['surgery?']=='2')&(horse_data['pulse']>(horse_data['pulse'].median()))&(horse_data['pain'].isna()), 'pain'] = '4'
horse_data.loc[(horse_data['pain'].isna()), 'pain'] = '3'
horse_data.head(5)

,surgery?,Age,rectal temperature,pulse,temperature of extremities,pain,outcome
0,2,1,38.5,66.0,3,5,2
1,1,1,39.2,88.0,4,3,3
2,2,1,38.3,40.0,1,3,1
3,1,2,39.1,164.0,4,2,2
4,2,1,37.3,104.0,1,5,2


In [ ]:
#we have no more gaps
horse_data.isna().sum()

surgery?                      0
Age                           0
rectal temperature            0
pulse                         0
temperature of extremities    0
pain                          0
outcome                       0
dtype: int64

In [ ]:
#the length of the database is 291 rows, that means that we have deleted just 9 rows, which is not so much
print(len(horse_data))

291
